In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Mount Isa,-20.7333,139.5000,76.77,47,0,12.66,AU,2021-07-28 00:41:29
1,1,Yellowknife,62.4560,-114.3525,62.33,48,40,3.00,CA,2021-07-28 00:38:15
2,2,Kapaa,22.0752,-159.3190,85.98,76,90,3.00,US,2021-07-28 00:38:27
3,3,Mahebourg,-20.4081,57.7000,67.24,100,75,14.97,MU,2021-07-28 00:41:30
4,4,Rikitea,-23.1203,-134.9692,74.12,78,100,14.72,PF,2021-07-28 00:41:30


In [3]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [5]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig


Figure(layout=FigureLayout(height='420px'))

In [6]:
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp)

In [7]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [10]:
heat_layer = gmaps.heatmap_layer(locations, weights=temps)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [11]:
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)


In [12]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [13]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [14]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Mount Isa,-20.7333,139.5000,76.77,47,0,12.66,AU,2021-07-28 00:41:29
2,2,Kapaa,22.0752,-159.3190,85.98,76,90,3.00,US,2021-07-28 00:38:27
8,8,Pandan,14.0453,124.1694,86.50,69,100,16.73,PH,2021-07-28 00:40:51
10,10,Dingle,10.9995,122.6711,85.03,73,75,16.11,PH,2021-07-28 00:41:32
12,12,Bambanglipuro,-7.9500,110.2833,78.21,76,78,6.44,ID,2021-07-28 00:41:32
17,17,Hamilton,39.1834,-84.5333,85.23,68,1,3.44,US,2021-07-28 00:38:50
23,23,Tabas,33.5959,56.9244,81.50,19,1,7.25,IR,2021-07-28 00:41:37
31,31,Krasnorechenskiy,44.6265,135.3532,84.16,45,91,3.24,RU,2021-07-28 00:41:41
35,35,Lago Da Pedra,-4.3333,-45.1667,77.20,56,21,2.48,BR,2021-07-28 00:41:43
40,40,Jamestown,42.0970,-79.2353,76.32,78,1,5.75,US,2021-07-28 00:41:45


In [15]:
preferred_cities_df.count()

City_ID       201
City          201
Lat           201
Lng           201
Max Temp      201
Humidity      201
Cloudiness    201
Wind Speed    201
Country       201
Date          201
dtype: int64

In [16]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Mount Isa,AU,76.77,-20.7333,139.5000,
2,Kapaa,US,85.98,22.0752,-159.3190,
8,Pandan,PH,86.50,14.0453,124.1694,
10,Dingle,PH,85.03,10.9995,122.6711,
12,Bambanglipuro,ID,78.21,-7.9500,110.2833,
17,Hamilton,US,85.23,39.1834,-84.5333,
23,Tabas,IR,81.50,33.5959,56.9244,
31,Krasnorechenskiy,RU,84.16,44.6265,135.3532,
35,Lago Da Pedra,BR,77.20,-4.3333,-45.1667,
40,Jamestown,US,76.32,42.0970,-79.2353,


In [17]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [25]:
#Iterate through the data frame
for index, row in hotel_df.iterrows():
    #Get the latitude and longitude
    lat = row['Lat']
    lng = row['Lng']
    
    #Add the latitude and longitude to location key for the params dictionary
    params['location'] = f"{lat},{lng}"
    
    #Use the search term: "lodging" and our latitude and longitude
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    #Make request and get the JSON data from the search
    hotels = requests.get(base_url, params=params).json()
    #Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
    except (IndexError):
        print('Hotel not found... skipping.')
    

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [26]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Mount Isa,AU,76.77,-20.7333,139.5000,ibis Styles Mt Isa Verona
2,Kapaa,US,85.98,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
8,Pandan,PH,86.50,14.0453,124.1694,Bonifacio Lodge
10,Dingle,PH,85.03,10.9995,122.6711,SEGAYA PROPERTY
12,Bambanglipuro,ID,78.21,-7.9500,110.2833,Puri Brata
17,Hamilton,US,85.23,39.1834,-84.5333,Six Acres Bed & Breakfast
23,Tabas,IR,81.50,33.5959,56.9244,شرکت معماری مهراز شایگان آراد
31,Krasnorechenskiy,RU,84.16,44.6265,135.3532,
35,Lago Da Pedra,BR,77.20,-4.3333,-45.1667,
40,Jamestown,US,76.32,42.0970,-79.2353,Hampton Inn & Suites Jamestown


In [27]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [28]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [29]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

#Store the data frame row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [30]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))